<a href="https://colab.research.google.com/github/Adrianus-Bagas/Hate-Speech-Detection/blob/main/Hate_Speech_Detection_(Preprocessing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deteksi Ujaran Kebencian Menggunakan Metode Neural Network

## 1. Import beberapa libraries

In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 4.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, LSTM, Bidirectional, GRU
from tensorflow.keras import optimizers
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 2. Memuat data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/hate_speech_dataset.csv')
df

,Unnamed: 0,Label,Tweet,hasil_praproses
0,0,Non_HS,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...,spardaxyz fadli zon mendagri nonaktif ahok gub...
1,1,Non_HS,RT @baguscondromowo: Mereka terus melukai aksi...,baguscondromowo luka aksi rangka penjara ahok ...
2,2,Non_HS,Sylvi: bagaimana gurbernur melakukan kekerasan...,sylvi gurbernur keras perempuan bukti foto bar...
3,3,Non_HS,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja...",ahmad dhani puas debat pilkada jalan bekas ungkap
4,4,Non_HS,RT @lisdaulay28: Waspada KTP palsu.....kawal P...,lisdaulay waspada ktp palsukawal pilkada
...,...,...,...,...
2695,2724,HS,Yagimanasih anjir gue dirumah doang dari pagi ...,yagimanasih anjir rumah doang pagi gangerti ar...
2696,2725,Non_HS,Yah sekian pandangan awam dari saya wkwwk seka...,yah sekian pandang awam wkwwk lg cuman pandang...
2697,2726,HS,yang butuh memek hubungi rp haneul terdekat',butuh memek hubung rp haneul dekat
2698,2727,HS,Yang sabar gua mah punya adek2 kunyuk yg bau k...,sabar gua mah adek kunyuk yg bau kek sempak kuda


Keterangan Label
1. Non_HS = Baik
2. HS = Buruk

## 3. Text Preprocessing

In [ ]:
df.isnull().sum()

Label    0
Tweet    0
dtype: int64

In [ ]:
df = df.drop_duplicates(subset=["Tweet"])

In [ ]:
df

,Label,Tweet
0,Non_HS,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...
1,Non_HS,RT @baguscondromowo: Mereka terus melukai aksi...
2,Non_HS,Sylvi: bagaimana gurbernur melakukan kekerasan...
3,Non_HS,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja..."
4,Non_HS,RT @lisdaulay28: Waspada KTP palsu.....kawal P...
...,...,...
2724,HS,Yagimanasih anjir gue dirumah doang dari pagi ...
2725,Non_HS,Yah sekian pandangan awam dari saya wkwwk seka...
2726,HS,yang butuh memek hubungi rp haneul terdekat'
2727,HS,Yang sabar gua mah punya adek2 kunyuk yg bau k...


In [ ]:
def case_folding(text):
  text = re.sub(r'[^a-zA-Z\s]','',text)
  text = re.sub('USER','',text)
  text = re.sub('RT','',text)
  text = re.sub('HTML','',text)
  text = re.sub(r"(?:\@|https?\://)\S+", "", text)
  text = re.sub(r"http\S+", "", text)
  text = re.sub('\n', '', text)
  text = re.sub('RT', '', text)
  text = re.sub("[^a-zA-Z^']", " ", text)
  text = re.sub(" {2,}", " ", text)
  text = text.strip()
  text = re.sub(r'\s+', ' ', text)
  return text.lower()

In [ ]:
kamus = pd.read_csv('/content/drive/MyDrive/dataset/kamusalay.csv')
kamus

,slang,asli
0,ajj,saja
1,akoh,aku
2,amaca,ah masa
3,amuh,kamu
4,aph,apa
...,...,...
233,yaw,ya
234,yoi,iya
235,yups,ya
236,yupz,ya


In [ ]:
kamus_alay = dict(zip(kamus['slang'], kamus['asli']))
kamus_alay

{'ajj': 'saja',
 'akoh': 'aku',
 'amaca': 'ah masa',
 'amuh': 'kamu',
 'aph': 'apa',
 'apose': 'apa',
 'apz': 'apa',
 'aq': 'saya',
 'baryaw': 'sabar ya',
 'baryw': 'sabar ya',
 'bryaw': 'sabar ya',
 'bryw': 'sabar ya',
 'bay': 'selamat tinggal',
 'besoq': 'besok',
 'beud': 'banget',
 'bhay': 'selamat tinggal',
 'bisya': 'bisa',
 'biza': 'bisa',
 'bntr': 'sebentar',
 'bokap': 'ayah',
 'bkap': 'ayah',
 'bkp': 'ayah',
 'bokaps': 'ayah',
 'bokapz': 'ayah',
 'bs': 'bisa',
 'bsa': 'bisa',
 'bsk': 'besok',
 'bsoq': 'besok',
 'bte': 'bosan',
 'bozen': 'bosan',
 'bozn': 'bosan',
 'bzn': 'bosan',
 'bzen': 'bosan',
 'cabut': 'pergi',
 'caiank': 'sayang',
 'cekola': 'sekolah',
 'cekolah': 'sekolah',
 'celalaw': 'selalu',
 'celalu': 'selalu',
 'cemungudh': 'semangat',
 'cemungut': 'semangat',
 'cemunguth': 'semangat',
 'cibuq': 'sibuk',
 'cini': 'sini',
 'ciyus': 'serius',
 'cll': 'selalu',
 'cllu': 'selalu',
 'cllw': 'selalu',
 'cpe': 'capek',
 'cpee': 'capek',
 'cewe': 'cewek',
 'cwe': 'cewek',


In [ ]:
def normalisasi(txt):
    return ' '.join([kamus_alay[word] if word in kamus_alay else word for word in txt.split(' ')])

In [ ]:
def hapus_stopwords(txt):
  stop_words = set(stopwords.words('indonesian'))
  return " ".join(['' if word in stop_words else word for word in txt.split(' ')])

In [ ]:
def stem(txt):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  return stemmer.stem(txt)

In [ ]:
def praproses(text):
  text = case_folding(text)
  text = normalisasi(text)
  text = hapus_stopwords(text)
  text = stem(text)
  return text

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
tweet = df.Tweet
hasil_praproses = tweet.apply(praproses)

In [ ]:
df['hasil_praproses']=hasil_praproses
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Label,Tweet,hasil_praproses
0,Non_HS,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...,spardaxyz fadli zon mendagri nonaktif ahok gub...
1,Non_HS,RT @baguscondromowo: Mereka terus melukai aksi...,baguscondromowo luka aksi rangka penjara ahok ...
2,Non_HS,Sylvi: bagaimana gurbernur melakukan kekerasan...,sylvi gurbernur keras perempuan bukti foto bar...
3,Non_HS,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja...",ahmad dhani puas debat pilkada jalan bekas ungkap
4,Non_HS,RT @lisdaulay28: Waspada KTP palsu.....kawal P...,lisdaulay waspada ktp palsukawal pilkada
...,...,...,...
2724,HS,Yagimanasih anjir gue dirumah doang dari pagi ...,yagimanasih anjir rumah doang pagi gangerti ar...
2725,Non_HS,Yah sekian pandangan awam dari saya wkwwk seka...,yah sekian pandang awam wkwwk lg cuman pandang...
2726,HS,yang butuh memek hubungi rp haneul terdekat',butuh memek hubung rp haneul dekat
2727,HS,Yang sabar gua mah punya adek2 kunyuk yg bau k...,sabar gua mah adek kunyuk yg bau kek sempak kuda


In [ ]:
df.to_csv('/content/drive/MyDrive/dataset/project 4.csv')